# Draw circuit diagrams

In [ ]:
# Install SchemDraw, https://schemdraw.readthedocs.io/en/latest/index.html; uncomment the line below to install
# !pip install git+https://bitbucket.org/cdelker/schemdraw.git@11f6777

In [ ]:
#%config InlineBackend.figure_format = 'svg'

%matplotlib inline
import schemdraw
import schemdraw.elements as elm
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import LFPy
import neuron
import brainsignals.plotting_convention as epc

## RC circuit diagram and Vm dynamics

In [ ]:
# generate soma response to step-current input
class Simulation(object):
    def run(self):
        soma = neuron.h.Section(name='soma')
        soma.L = 20
        soma.diam = 10

        morphology = neuron.h.SectionList([soma])
        
        cell = LFPy.Cell(morphology=morphology, 
                         cm=1,  # μF/cm2
                         Ra=100,  # (ohm cm)
                         v_init=-65,  # (mV)
                         passive=True, 
                         passive_parameters=dict(
                             g_pas=0.0000333,  # (S/cm2)
                             e_pas=-65,),  # (mV)
                         nsegs_method=None,
                         delete_sections=False,
                         dt=2**-4,  # (ms)
                         tstop=500  # (ms)
                        )

        stimulus = LFPy.StimIntElectrode(cell, 
                                         idx=0, 
                                         pptype='IClamp', 
                                         record_current=True, 
                                         record_potential=True, 
                                         amp=0.005, dur=200, delay=100)
        
        cell.simulate()
        
        return cell.tvec, stimulus.i, stimulus.v

sim = Simulation()
t, i, v = sim.run()
    
fig, (ax0, ax1) = plt.subplots(2, 1, sharex=True)
ax0.plot(t, i)
ax1.plot(t, v)

In [ ]:
fig = plt.figure(figsize=(6, 2.5))
gs = GridSpec(2, 2)

axes = []
ax = fig.add_subplot(gs[:, 0])
axes.append(ax)
ax.axis('off')

d = schemdraw.Drawing(backend='matplotlib')
d.config(unit=3, fontsize=8, lw=1.5)
d += elm.Line().left().length(1.5)
d += elm.Capacitor().up().label('$c_\mathrm{m}$')
d += elm.Line().right()
d += elm.Resistor().down().length(1.5).label(r'$\overline{g}_\mathrm{L}$')
d += elm.BatteryCell().down().length(1.5).label(r'$E_\mathrm{L}$')
d += elm.Line().left().length(1.5)
d += elm.Line().down().length(1)
d += elm.Dot().label('$V_\mathrm{i}$', loc='bottom').label('intracellular', loc='left')
d += elm.Line().right()
d += elm.SourceI().up().length(5).reverse().label(r'$I_\mathrm{ext}/A$', loc='bottom')
d += elm.Line().left()
d += elm.Dot().label('$V_\mathrm{e}$').label('extracellular', loc='left')
d += elm.Line().down().length(1)
d.draw(ax=ax)
ax.set_aspect('equal')
ax.set_xlim(-2.5, 4)
ax.set_ylim(-1.5, 5)

# plot RC-curcuit responses
sim = Simulation()
t, i, v = sim.run()

ax = fig.add_subplot(gs[0, 1])
axes.append(ax)
ax.plot(t, i, c='k')
ax.set_ylabel('$I_\mathrm{ext}$ (nA)')

ax = fig.add_subplot(gs[1, 1])
axes.append(ax)
ax.plot(t, v, c='k')
ax.set_ylim(-70, -20)
arrowprops = dict(
    arrowstyle="->",
    connectionstyle="arc3,rad=0.2")
ax.annotate(r'$\tau_\mathrm{m}=c_\mathrm{m}\//\/\overline{g}_\mathrm{L}$', 
            xy=(150, v[t==150]), 
            xytext=(150, v[t==150]+20),
            arrowprops=arrowprops)
ax.set_xlabel('$t$ (ms)')
ax.set_ylabel('$V_\mathrm{m}$ (mV)')

epc.mark_subplots(axes[0], ypos=1)
epc.mark_subplots(axes[1:], letters='BC')
epc.simplify_axes(axes)

fig.savefig('RC_circuit.pdf', bbox_inches='tight')

# HH circuit and simulation

In [ ]:
# generate soma response to step-current input
class HHSimulation(object):
    def run(self):
        soma = neuron.h.Section(name='soma')
        soma.L = 20
        soma.diam = 10
        soma.insert('hh')

        morphology = neuron.h.SectionList([soma])
        
        cell = LFPy.Cell(morphology=morphology, 
                         cm=1,  # (μF/cm2)
                         Ra=100,  # (ohm cm)
                         v_init=-65,  # (mV)
                         passive=False, 
                         nsegs_method=None,
                         delete_sections=False,
                         dt=2**-4,  # (ms)
                         tstop=50  # (ms)
                        )

        stimulus = LFPy.StimIntElectrode(cell, 
                                         idx=0, 
                                         pptype='IClamp', 
                                         record_current=True, 
                                         record_potential=True, 
                                         amp=0.1, dur=20, delay=10)
        
        cell.simulate()
        
        return cell.tvec, stimulus.i, stimulus.v

sim = HHSimulation()
t, i, v = sim.run()
    
fig, (ax0, ax1) = plt.subplots(2, 1, sharex=True)
ax0.plot(t, i)
ax1.plot(t, v)

In [ ]:
fig = plt.figure(figsize=(6, 2.5))
gs = GridSpec(2, 3, wspace=0.2)

axes = []
ax = fig.add_subplot(gs[:, :2])
axes.append(ax)
ax.axis('off')

d = schemdraw.Drawing(backend='matplotlib')
d.config(unit=3, fontsize=8, lw=1.5)
d.push()
d += elm.Capacitor().up().label('$c_\mathrm{m}$')
d += elm.Line().right().length(1.5)
d.push()
d += elm.Resistor().down().reverse().length(1.5).label(r'$\overline{g}_\mathrm{L}$')
d += elm.BatteryCell().down().length(1.5).label(r'$E_\mathrm{L}$')
d.pop()
d += elm.Line().right().length(1.5)
d.push()
d += elm.ResistorVar().down().length(1.5).reverse().label('$g_\mathrm{Na}$', ofst=(0, 0.2))
d += elm.BatteryCell().down().length(1.5).label(r'$E_\mathrm{Na}$')
d.pop()
d += elm.Line().right().length(1.5)
d += elm.ResistorVar().down().length(1.5).reverse().label(r'$g_\mathrm{K}$', ofst=(0, 0.2))
d += elm.BatteryCell().down().length(1.5).label(r'$E_\mathrm{K}$')
d += elm.Line().left().length(2.25)
d.pop()
d += elm.Line().right().length(2.25)
d += elm.Line().down().length(1)
d += elm.Dot().label('intracellular', loc='left').label('$V_\mathrm{i}$', loc='bottom')
d += elm.Line().right().length(3.5)
d += elm.SourceI().up().length(5).reverse().label(r'$I_\mathrm{ext}/A$', loc='bottom')
d += elm.Line().left().length(3.5)
d += elm.Dot().label('$V_\mathrm{e}$').label('extracellular', loc='left')
d += elm.Line().down().length(1)

d.draw(ax=ax)
ax.set_aspect('equal')
ax.set_xlim(-1, 7.)
ax.set_ylim(-1.5, 4.5)


# plot RC-curcuit responses
sim = HHSimulation()
t, i, v = sim.run()

ax = fig.add_subplot(gs[0, 2])
axes.append(ax)
ax.plot(t, i, c='k')
ax.set_ylabel('$I_\mathrm{ext}$ (nA)')

ax = fig.add_subplot(gs[1, 2])
axes.append(ax)
ax.plot(t, v, c='k')
ax.set_xlabel('$t$ (ms)')
ax.set_ylabel('$V_\mathrm{m}$ (mV)')

epc.mark_subplots(axes[0], ypos=1)
epc.mark_subplots(axes[1:], letters='BC')
epc.simplify_axes(axes)

fig.savefig('HH_circuit.pdf', bbox_inches='tight')